In [ ]:
# ========================================
# PATH CONFIGURATION
# ========================================
# Modify these values according to your configuration

USER_PATH = 'users/your_username'              # Your personal user path in GEE
PROJECT_PATH = 'projects/your-project/assets'  # Your project path in GEE

# Usage examples:
# USER_PATH = 'users/cnav'
# PROJECT_PATH = 'projects/ee-ugr-default-project/assets'

## 1. Authentication and Initialization

### 1.1 Basic authentication

In [ ]:
import ee

# Forced authentication (opens browser window)
ee.Authenticate(force=True)
ee.Initialize()

print("Authentication successful")

### 1.2 Authentication with full permissions

If you need additional permissions for copy/delete operations, use this option with all scopes.

In [ ]:
import ee

ee.Authenticate(
    force=True,
    scopes=[
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/devstorage.full_control',
        'https://www.googleapis.com/auth/cloud-platform'
    ]
)
ee.Initialize()

print("Authentication with full permissions successful")

## 2. Query Operations

### 2.1 List assets in a location

In [ ]:
import ee
ee.Initialize()

# List assets in personal user
assets_user = ee.data.listAssets({'parent': USER_PATH})
print(f"Assets in {USER_PATH}:")
for asset in assets_user.get('assets', []):
    print(f"  - {asset['id']} ({asset['type']})")

# List assets in project
assets_project = ee.data.listAssets({'parent': PROJECT_PATH})
print(f"\nAssets in {PROJECT_PATH}:")
for asset in assets_project.get('assets', []):
    print(f"  - {asset['id']} ({asset['type']})")

### 2.2 Check asset information

In [ ]:
import ee
ee.Initialize()

# Check information for a specific asset
asset_id = f'{USER_PATH}/asset_name'

try:
    info = ee.data.getAsset(asset_id)
    print(f"Type: {info['type']}")
    print(f"ID: {info['id']}")
    print(f"Name: {info['name']}")
except Exception as e:
    print(f"Error getting information: {e}")

### 2.3 Check asset permissions

In [ ]:
import ee
ee.Initialize()

# Check ACL (Access Control List) of an asset
asset_id = f'{USER_PATH}/asset_name'

try:
    acl = ee.data.getAssetAcl(asset_id)
    print("Asset permissions:")
    print(acl)
except Exception as e:
    print(f"Error getting permissions: {e}")

## 3. Copy Assets - Individual Operations

### 3.1 Copy an individual image

In [ ]:
import ee
ee.Initialize()

# Copy an image from one place to another
src_image = f'{USER_PATH}/image_name'
dst_image = f'{PROJECT_PATH}/destination_image_name'

print(f"Copying image:\n  {src_image}\n  → {dst_image}")

try:
    ee.data.copyAsset(src_image, dst_image)
    print("Copy completed successfully")
except Exception as e:
    print(f"Error copying: {e}")

### 3.2 Copy an individual table

In [ ]:
import ee
ee.Initialize()

# Copy a table (FeatureCollection)
src_table = f'{USER_PATH}/table_name'
dst_table = f'{PROJECT_PATH}/destination_table_name'

print(f"Copying table:\n  {src_table}\n  → {dst_table}")

try:
    ee.data.copyAsset(src_table, dst_table)
    print("Copy completed successfully")
except Exception as e:
    print(f"Error copying: {e}")

### 3.3 Copy multiple images from a folder

In [ ]:
import ee
ee.Initialize()

# Copy all images from a source folder to a destination folder
src_folder = f'{USER_PATH}/source_folder'
dst_folder = f'{PROJECT_PATH}/destination_folder'

# List assets in the source folder
assets = ee.data.listAssets({'parent': src_folder})['assets']

print(f"Copying {len(assets)} assets from {src_folder} to {dst_folder}\n")

for asset in assets:
    src = asset['id']
    asset_name = src.split('/')[-1]
    dst = f'{dst_folder}/{asset_name}'
    
    print(f"Copying: {asset_name}")
    try:
        ee.data.copyAsset(src, dst)
        print(f"  → Success")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nProcess completed")

## 4. Copy ImageCollections

### 4.1 Copy complete ImageCollection (same location type)

To copy between users or between projects, first create the destination collection and then copy the images.

In [ ]:
import ee
ee.Initialize()

# Define source and destination
src_collection = f'{USER_PATH}/source_collection'
dst_collection = f'{USER_PATH}/destination_collection'

# 1. Create the destination ImageCollection
print(f"Creating ImageCollection: {dst_collection}")
try:
    ee.data.createAsset({'type': 'ImageCollection'}, dst_collection)
    print("  → Collection created")
except Exception as e:
    print(f"  → Error (may already exist): {e}")

# 2. Copy all images
print(f"\nCopying images from {src_collection}...")
imgs = ee.ImageCollection(src_collection).toList(10000)
n = imgs.size().getInfo()
print(f"Total images to copy: {n}\n")

for i in range(n):
    img = ee.Image(imgs.get(i))
    src = img.getInfo()['id']
    img_name = src.split('/')[-1]
    dst = f'{dst_collection}/{img_name}'
    
    print(f"  [{i+1}/{n}] Copying: {img_name}")
    try:
        ee.data.copyAsset(src, dst)
        print(f"    → Success")
    except Exception as e:
        print(f"    → Error: {e}")

print("\nCollection copy completed")

### 4.2 Copy ImageCollection to folder (without creating collection)

Useful for copying images from a collection to a project folder.

In [ ]:
import ee
ee.Initialize()

# Copy images from an ImageCollection to a folder
src_collection = f'{USER_PATH}/source_collection'
dst_folder = f'{PROJECT_PATH}/destination_folder'

# Get list of images
image_list = ee.ImageCollection(src_collection).toList(10000)
count = image_list.size().getInfo()
print(f"Total images: {count}\n")

for i in range(count):
    img = ee.Image(image_list.get(i))
    info = img.getInfo()
    
    src_id = info['id']
    img_name = src_id.split('/')[-1]
    dst_id = f"{dst_folder}/{img_name}"

    print(f"[{i+1}/{count}] Copying: {img_name}")
    try:
        ee.data.copyAsset(src_id, dst_id)
        print(f"  → Success")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nCopy completed")

### 4.3 Copy ImageCollection to project (creating new collection)

Creates a new ImageCollection in the project and copies all images.

In [ ]:
import ee
ee.Initialize()

# Copy complete ImageCollection to project
src_collection = f'{USER_PATH}/source_collection'
dst_collection = f'{PROJECT_PATH}/destination_collection'

# 1. Create the ImageCollection in the project
print(f"Creating ImageCollection: {dst_collection}")
try:
    ee.data.createAsset({'type': 'ImageCollection'}, dst_collection)
    print("  → Collection created successfully")
except Exception as e:
    print(f"  → Warning: {e}")

# 2. Copy all images
print(f"\nCopying images...")
image_list = ee.ImageCollection(src_collection).toList(10000)
count = image_list.size().getInfo()
print(f"Total: {count} images\n")

for i in range(count):
    img = ee.Image(image_list.get(i))
    info = img.getInfo()
    
    src_id = info['id']
    img_name = src_id.split('/')[-1]
    dst_id = f"{dst_collection}/{img_name}"

    print(f"[{i+1}/{count}] {img_name}")
    try:
        ee.data.copyAsset(src_id, dst_id)
        print(f"  → Copied")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nProcess completed")

## 5. Copy Complete Structures (Recursive)

### 5.1 Recursive copy function

This function copies complete folders with all their subfolders, ImageCollections, images and tables.

In [ ]:
import ee
ee.Initialize()

def exists(asset_id):
    """Check if an asset exists."""
    try:
        ee.data.getAsset(asset_id)
        return True
    except:
        return False

def join_path(parent, child_name):
    """Build a valid EE asset path."""
    return parent.rstrip('/') + '/' + child_name.split('/')[-1]

def copy_asset_recursive(src, dst):
    """
    Recursively copy a GEE asset (folder, collection, image or table).
    
    Args:
        src: Source asset path
        dst: Destination asset path
    """
    info = ee.data.getAsset(src)
    asset_type = info["type"]
    
    print(f"\nProcessing: {src} ({asset_type})")

    # ---- FOLDERS ----
    if asset_type == "FOLDER":
        if not exists(dst):
            print(f"  → Creating folder: {dst}")
            ee.data.createFolder(dst)
        else:
            print(f"  → Folder already exists: {dst}")

        children = ee.data.listAssets({"parent": src}).get("assets", [])
        print(f"  → Found {len(children)} items")
        
        for child in children:
            child_src = child["name"]
            child_dst = join_path(dst, child_src)
            copy_asset_recursive(child_src, child_dst)

    # ---- IMAGE COLLECTION ----
    elif asset_type == "IMAGE_COLLECTION":
        if not exists(dst):
            print(f"  → Creating ImageCollection: {dst}")
            ee.data.createAsset({"type": "IMAGE_COLLECTION"}, dst)
        else:
            print(f"  → ImageCollection already exists: {dst}")

        children = ee.data.listAssets({"parent": src}).get("assets", [])
        print(f"  → Copying {len(children)} images")
        
        for i, child in enumerate(children, 1):
            child_src = child["name"]
            child_dst = join_path(dst, child_src)
            print(f"     [{i}/{len(children)}] Copying image: {child_src.split('/')[-1]}")
            try:
                ee.data.copyAsset(child_src, child_dst)
            except Exception as e:
                print(f"       Error: {e}")

    # ---- IMAGE / TABLE / OTHER ----
    else:
        print(f"  → Copying {asset_type}: {src}")
        try:
            ee.data.copyAsset(src, dst)
            print(f"     Success")
        except Exception as e:
            print(f"     Error: {e}")

print("Recursive copy function defined")

### 5.2 Use the recursive copy function

In [ ]:
import ee
ee.Initialize()

# Define source and destination
src_folder = f'{USER_PATH}/complete_folder'
dst_folder = f'{PROJECT_PATH}/complete_folder_copy'

print(f"Starting recursive copy:")
print(f"  Source: {src_folder}")
print(f"  Destination: {dst_folder}")
print("="*60)

try:
    copy_asset_recursive(src_folder, dst_folder)
    print("\n" + "="*60)
    print("Recursive copy completed successfully")
except Exception as e:
    print(f"\nError during copy: {e}")

## 6. Delete Assets - Individual Operations

### 6.1 Delete an individual image or table

In [ ]:
import ee
ee.Initialize()

# Delete an individual asset (image or table)
asset_to_delete = f'{USER_PATH}/asset_name'

print(f"Deleting: {asset_to_delete}")

try:
    ee.data.deleteAsset(asset_to_delete)
    print("Asset deleted successfully")
except Exception as e:
    print(f"Error deleting: {e}")

### 6.2 Delete an empty ImageCollection

In [ ]:
import ee
ee.Initialize()

# Delete an ImageCollection that is empty
collection_id = f'{USER_PATH}/empty_collection'

print(f"Deleting empty collection: {collection_id}")

try:
    ee.data.deleteAsset(collection_id)
    print("Collection deleted successfully")
except Exception as e:
    print(f"Error: {e}")

### 6.3 Delete ImageCollection with all its images


In [ ]:
import ee
ee.Initialize()

collection_id = f'{PROJECT_PATH}/collection_with_images'

print(f"Deleting ImageCollection: {collection_id}")

# 1. Get list of images
try:
    image_list = ee.ImageCollection(collection_id).toList(10000)
    count = image_list.size().getInfo()
    print(f"Total images: {count}\n")

    # 2. Delete each image
    for i in range(count):
        # Always get the image at index 0 since the list shrinks after each deletion
        img = ee.Image(image_list.get(0))
        img_id = img.getInfo()['id']
        print(f"[{i+1}/{count}] Deleting image: {img_id}")
        try:
            ee.data.deleteAsset(img_id)
            print("  → Deleted")
            # Update the list after deletion
            image_list = ee.ImageCollection(collection_id).toList(10000)
        except Exception as e:
            print(f"  → Error: {e}")

    # 3. Delete the empty collection
    print(f"\nDeleting empty collection: {collection_id}")
    ee.data.deleteAsset(collection_id)
    print("Deletion complete")
    
except Exception as e:
    print(f"Error: {e}")

### 6.4 Delete multiple assets from a folder

In [ ]:
import ee
ee.Initialize()

# Delete all assets from a folder (without deleting the folder)
folder_path = f'{USER_PATH}/folder_to_clean'

print(f"Deleting assets from: {folder_path}\n")

try:
    assets = ee.data.listAssets({'parent': folder_path})['assets']
    print(f"Total assets to delete: {len(assets)}\n")
    
    for i, asset in enumerate(assets, 1):
        asset_id = asset['id']
        asset_type = asset['type']
        print(f"[{i}/{len(assets)}] Deleting {asset_type}: {asset_id}")
        
        try:
            ee.data.deleteAsset(asset_id)
            print("  → Deleted")
        except Exception as e:
            print(f"  → Error: {e}")
    
    print("\nProcess completed")
    
except Exception as e:
    print(f"Error: {e}")

## 7. Delete Complete Structures (Recursive)

### 7.1 Basic recursive deletion function

In [ ]:
import ee
ee.Initialize()

def delete_recursive(asset_path):
    """
    Delete a GEE asset recursively.
    
    Args:
        asset_path: Path of the asset to delete (folder, collection, etc.)
    """
    
    # List asset contents
    children = ee.data.listAssets({'parent': asset_path}).get('assets', [])
    
    print(f"\nProcessing: {asset_path}")
    print(f"  → {len(children)} items found")

    for child in children:
        child_id = child["id"]
        asset_type = child["type"]

        print(f"\n  Item: {child_id} ({asset_type})")

        if asset_type == "FOLDER":
            # Folder: delete recursively
            delete_recursive(child_id)

        elif asset_type == "IMAGE_COLLECTION":
            print(f"    → Deleting images from collection...")

            # Get list of images
            imgs = ee.data.listAssets({'parent': child_id}).get('assets', [])
            print(f"    → {len(imgs)} images to delete")

            for i, img in enumerate(imgs, 1):
                img_id = img["id"]
                print(f"       [{i}/{len(imgs)}] Deleting: {img_id.split('/')[-1]}")
                try:
                    ee.data.deleteAsset(img_id)
                except Exception as e:
                    print(f"         Error: {e}")

            print(f"    → Deleting empty collection: {child_id}")
            ee.data.deleteAsset(child_id)

        elif asset_type in ["IMAGE", "TABLE"]:
            print(f"    → Deleting {asset_type.lower()}: {child_id}")
            try:
                ee.data.deleteAsset(child_id)
            except Exception as e:
                print(f"       Error: {e}")

        else:
            print(f"    → Unhandled type: {asset_type}")

    # Finally delete the container asset
    print(f"\n  → Deleting container: {asset_path}")
    try:
        ee.data.deleteAsset(asset_path)
        print(f"     Success")
    except Exception as e:
        print(f"     Error: {e}")

print("Recursive deletion function defined")

### 7.2 Use the recursive deletion function

In [ ]:
import ee
ee.Initialize()

# WARNING: This operation is irreversible
# Make sure you want to delete all contents

target_folder = f'{USER_PATH}/folder_to_delete'

print("="*60)
print("WARNING: Recursive deletion")
print("="*60)
print(f"Will delete: {target_folder}")
print("This operation is IRREVERSIBLE")
print("="*60)

# Uncomment the following line to execute
# delete_recursive(target_folder)
# print("\nRECURSIVE DELETION COMPLETED")

print("\n(Code commented for safety)")

### 7.3 Recursive deletion function with progress bar

Enhanced version with visual feedback using tqdm.

In [ ]:
import ee
ee.Initialize()

# Install tqdm if not available
try:
    from tqdm import tqdm
except ImportError:
    print("Installing tqdm...")
    import sys
    !{sys.executable} -m pip install tqdm
    from tqdm import tqdm

def delete_recursive_progress(asset_path, level=0):
    """
    Recursively delete GEE assets with progress bar.
    
    Args:
        asset_path: Path of the asset to delete
        level: Depth level (for indentation)
    """
    
    # Get assets within the folder/collection
    children = ee.data.listAssets({'parent': asset_path}).get('assets', [])

    indent = "  " * level
    print(f"{indent}Exploring: {asset_path} ({len(children)} assets)")

    # Progress bar for contents
    for child in tqdm(children, desc=f"{indent}Level {level}", leave=True):
        child_id = child["id"]
        asset_type = child["type"]

        print(f"{indent}→ {child_id.split('/')[-1]} ({asset_type})")

        if asset_type == "FOLDER":
            # Folder: recursion
            delete_recursive_progress(child_id, level + 1)

        elif asset_type == "IMAGE_COLLECTION":
            print(f"{indent}   → Deleting images...")

            imgs = ee.data.listAssets({'parent': child_id}).get('assets', [])

            # Progress bar for images
            for img in tqdm(imgs, desc=f"{indent}   Images", leave=False):
                img_id = img["id"]
                try:
                    ee.data.deleteAsset(img_id)
                except Exception as e:
                    print(f"{indent}      Error: {e}")

            print(f"{indent}   → Deleting empty collection")
            ee.data.deleteAsset(child_id)

        elif asset_type in ["IMAGE", "TABLE"]:
            try:
                ee.data.deleteAsset(child_id)
            except Exception as e:
                print(f"{indent}   Error: {e}")

        else:
            print(f"{indent}   Unhandled type: {asset_type}")

    # Delete the container
    print(f"{indent}→ Deleting: {asset_path}")
    try:
        ee.data.deleteAsset(asset_path)
    except Exception as e:
        print(f"{indent}  Error: {e}")

print("Deletion function with progress defined")

### 7.4 Use the deletion function with progress

In [ ]:
import ee
ee.Initialize()

# WARNING: This operation is irreversible
target_folder = f'{USER_PATH}/folder_to_delete'

print("="*60)
print("RECURSIVE DELETION WITH PROGRESS")
print("="*60)
print(f"Target: {target_folder}")
print("="*60)

# Uncomment to execute
# delete_recursive_progress(target_folder)
# print("\n✓ PROCESS COMPLETED")

print("\n(Code commented for safety)")

## 8. Additional Utilities

### 8.1 Create folder

In [ ]:
import ee
ee.Initialize()

# Create a new folder
new_folder = f'{PROJECT_PATH}/new_folder'

print(f"Creating folder: {new_folder}")

try:
    ee.data.createFolder(new_folder)
    print("Folder created successfully")
except Exception as e:
    print(f"Error (may already exist): {e}")

### 8.2 Create empty ImageCollection

In [ ]:
import ee
ee.Initialize()

# Create an empty ImageCollection
new_collection = f'{PROJECT_PATH}/new_collection'

print(f"Creating ImageCollection: {new_collection}")

try:
    ee.data.createAsset({'type': 'ImageCollection'}, new_collection)
    print("ImageCollection created successfully")
except Exception as e:
    print(f"Error (may already exist): {e}")

### 8.3 Check if an asset exists

In [ ]:
import ee
ee.Initialize()

def asset_exists(asset_id):
    """Check if an asset exists in GEE."""
    try:
        ee.data.getAsset(asset_id)
        return True
    except:
        return False

# Usage example
asset_to_check = f'{USER_PATH}/asset_name'

if asset_exists(asset_to_check):
    print(f"The asset exists: {asset_to_check}")
else:
    print(f"The asset does NOT exist: {asset_to_check}")

### 8.4 Count items in a folder or collection

In [ ]:
import ee
ee.Initialize()

def count_assets(parent_path):
    """Count assets in a folder or collection."""
    try:
        assets = ee.data.listAssets({'parent': parent_path})['assets']
        return len(assets)
    except Exception as e:
        print(f"Error: {e}")
        return 0

# Usage example
path_to_count = f'{USER_PATH}/folder'
count = count_assets(path_to_count)
print(f"Total assets in {path_to_count}: {count}")

### 8.5 List all asset types in a location

In [ ]:
import ee
ee.Initialize()

def list_assets_by_type(parent_path):
    """List and group assets by type."""
    try:
        assets = ee.data.listAssets({'parent': parent_path})['assets']
        
        by_type = {}
        for asset in assets:
            asset_type = asset['type']
            if asset_type not in by_type:
                by_type[asset_type] = []
            by_type[asset_type].append(asset['id'])
        
        print(f"Assets in {parent_path}:\n")
        for asset_type, items in by_type.items():
            print(f"{asset_type}: {len(items)}")
            for item in items:
                print(f"  - {item.split('/')[-1]}")
            print()
        
        return by_type
    
    except Exception as e:
        print(f"Error: {e}")
        return {}

# Usage example
list_assets_by_type(USER_PATH)

## 9. Complete Use Case Examples

### 9.1 Migrate everything from user to project

In [ ]:
import ee
ee.Initialize()

# Migrate a complete folder from user to project
src = f'{USER_PATH}/data_to_migrate'
dst = f'{PROJECT_PATH}/migrated_data'

print("Starting complete migration")
print(f"Source: {src}")
print(f"Destination: {dst}")
print("="*60)

try:
    # Copy everything recursively
    copy_asset_recursive(src, dst)
    print("\n" + "="*60)
    print("Migration completed successfully")
    print("\nNOTE: Original data has NOT been deleted.")
    print("If you want to delete it, use delete_recursive()")
    
except Exception as e:
    print(f"\nError during migration: {e}")

### 9.2 Backup assets before deleting

In [ ]:
import ee
ee.Initialize()

# Create backup before deleting
original = f'{USER_PATH}/important_data'
backup = f'{USER_PATH}/important_data_backup'

print("Creating backup...")
print(f"Original: {original}")
print(f"Backup: {backup}")
print("="*60)

try:
    # Copy to backup
    copy_asset_recursive(original, backup)
    print("\nBackup created successfully")
    print(f"\nNow you can safely delete {original}")
    print(f"The backup is at: {backup}")
    
except Exception as e:
    print(f"\nError creating backup: {e}")

### 9.3 Reorganize folder structure

In [ ]:
import ee
ee.Initialize()

# Reorganize by copying to new structure and then deleting the old one
old_structure = f'{USER_PATH}/old_structure'
new_structure = f'{PROJECT_PATH}/new_structure'

print("Reorganizing structure...")
print(f"From: {old_structure}")
print(f"To: {new_structure}")
print("="*60)

try:
    # 1. Copy to new location
    print("\n1. Copying to new structure...")
    copy_asset_recursive(old_structure, new_structure)
    
    print("\n2. Copy completed")
    print("\nNOTE: The old structure has NOT been deleted.")
    print("Verify that everything is correct in the new location")
    print("before deleting the old one.")
    print(f"\nTo delete: delete_recursive('{old_structure}')")
    
except Exception as e:
    print(f"\nError: {e}")

---

## Important Notes

### Limitations and Considerations

1. **API Limits**: GEE has limits on the number of operations per minute. If working with many assets, consider adding pauses between operations.

2. **Permissions**: Make sure you have the necessary permissions for both source and destination.

3. **Irreversible operations**: Deletions are permanent. Always make backups before deleting important data.

4. **Large ImageCollections**: For collections with more than 10,000 images, consider using pagination.

5. **Relative paths**: Paths in GEE must always be absolute (include the complete prefix).

### Best Practices

1. **Test first**: Before massive operations, test with a small subset.

2. **Backups**: Always backup before deleting or moving important data.

3. **Verify destinations**: Make sure destination folders/collections exist before copying.

4. **Monitor quotas**: Regularly check your GEE storage quotas.

5. **Use variables**: Define common paths at the beginning of the notebook to avoid typos.

### Support

For more information about the Google Earth Engine API:
- Official documentation: https://developers.google.com/earth-engine
- Asset management guide: https://developers.google.com/earth-engine/guides/asset_manager